## Practising tensor manipulation

This notebook contains excercises to get comfortable with tensor arithmetic in Pytorch. Run the cells from top to bottom and fill the cells with your code when asked to. 

Each task can be done in multiple ways. In particular it's possible to do with a single line using only pytorch functions, with no loops or extra variables. Try to solve it that way if you can.

You don't need a GPU to complete it, you just need fastai2 and pytorch installed.

References:
* [Pytorch tensor docs](https://pytorch.org/docs/stable/tensors.html)
* [Broadcasting](https://pytorch.org/docs/stable/notes/broadcasting.html)


In [ ]:
%load_ext autoreload
%autoreload 2

from fastai2.vision.all import *
from L3_solutions import *

In [ ]:
img = Image.open('samples/kitten.jpg')
img

In [ ]:
img_tensor = tensor(img)
img_tensor.shape

### Task 1
The image above has 408 rows and 612 columns and three channels. Storing them in that order is called HWC format (Height, Width, Channel). Rearange the data so the channel comes first (CHW). Your new shape should be \[3,408,612\]

In [ ]:
new_tensor = ... # put your code here

In [ ]:
#test,
assert new_tensor.equal(sol1(img_tensor))

In [ ]:
#Spoiler, run this for a sample solution
sol1??

### Task 2
Turn the image into a grayscale by taking the average across the 3 channels for every pixel

In [ ]:
gray_tensor = ... # put your code here

In [ ]:
#test, you should see a gray image of the cat
test_eq(gray_tensor.shape, img_tensor.shape[:-1])
show_image(gray_tensor, cmap='gray')

In [ ]:
#Spoiler, run this for a sample solution
sol2??

### Task 3
Turn the grayscale tensor into a rank 1 tensor which concatenates the columns left to right, top to bottom. For example: 

```tensor([[1, 2],
        [3, 4])```

should become `[1, 3, 2, 4]`

In [ ]:
dim1_tensor = ...# put your code here

In [ ]:
#test
assert len(dim1_tensor.shape) == 1 and dim1_tensor.shape[0] == gray_tensor.shape[0] * gray_tensor.shape[1]
test_eq(dim1_tensor, sol3(gray_tensor))

In [ ]:
#Spoiler, run this for a sample solution
sol3??

### Task 4
A simple way to downsample an image would be to divide it into 2x2 tiles and keep just top left pixel of each tile.
for example:
```
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
```
becomes
```
tensor([[ 1,  3],
        [ 9, 11]])
```
Downsample the `img_tensor` to get a tensor four times smaller still showing the same image of a kitten

In [ ]:
downsampled = ...# put your code here


In [ ]:
#test
assert downsampled.shape == (img_tensor.shape[0]//2, img_tensor.shape[1]//2,3)

show_image(downsampled)

In [ ]:
#Spoiler, run this for a sample solution
sol4??

### Task 5
A simple way to upsample an image would be to replace each pixel with a 2x2 square with four copies of that pixel. Upsample the `img_tensor` to get a tensor four times larger still showing the same image of a kitten

In [ ]:
upsampled = ...# put your code here

In [ ]:
#test
assert upsampled.shape == (img_tensor.shape[0]*2, img_tensor.shape[1]*2,3)

show_image(upsampled)

In [ ]:
#Spoiler, run this for a sample solution
sol5??

### Task 6
In the lesson Jeremy showed a simple model where a digit is classified by comparing to the mean of all threes and the mean of all sevens.

Another idea is for the given image to find which of the other images is the most similar to it, and use that as its class. Implement the `most_similar` function bellow that for a given image finds the index of the most similar image in terms of [Manhattan (L1) distance](https://en.wikipedia.org/wiki/Taxicab_geometry)

In [ ]:
#load data into tensors
def load_from(path): return torch.stack([tensor(Image.open(o)) for o in path.ls()])
path = untar_data(URLs.MNIST_SAMPLE)
train_threes  = load_from(path/'train'/'3')
train_sevens  = load_from(path/'train'/'7')
valid_threes = load_from(path/'valid'/'3')
valid_sevens = load_from(path/'valid'/'7')
train_all = torch.cat((train_threes,train_sevens))


In [ ]:
#return the index in dataset with the most similar element
def most_similar(x, dataset): ...# put your code here

In [ ]:
#test, you should see aover 97% accuracy here. It takes a while to run
correct_threes = [most_similar(tt,train_all) < len(train_threes) for tt in valid_threes]
correct_sevens = [most_similar(tt,train_all) >= len(train_threes) for tt in valid_sevens]
acc = 100 * tensor(correct_threes + correct_sevens).float().mean()
f'Your model has {acc:.2f}% accuracy!'

In [ ]:
#Spoiler, run this for a sample solution
sol6??

### Bonus
The task 6 solution classifies one image at a time in a loop. Try to change it to find nearest indices for a whole batch of images at once.
It should give the same result, but much faster. Especially on a GPU